In [2]:
import pandas as pd
import numpy as np
import pycountry
import zipcodes
from geopy.geocoders import Nominatim

  Using cached pycountry-22.3.5.tar.gz (10.1 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681896 sha256=0e54fa1d41891b25432b148e892ce9635842b99f318e7e21ccc2d41a78283c2e
  Stored in directory: c:\users\rafae\appdata\local\pip\cache\wheels\cd\29\8b\617685ed7942656b36efb06ff9247dbe832e3f4f7724fffc09
Successfully built pycountry


In [3]:
# Read data
df_events = pd.read_csv('../Natural-Disasters.csv',delimiter=';')
df_location_data = pd.read_csv('../pend-gdis-1960-2018-disasterlocations.csv',delimiter=',')

C:\Users\rafae\AppData\Local\Temp\ipykernel_38300\1686743832.py:2: DtypeWarning: Columns (18,19,24,25,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df_events = pd.read_csv('../Natural-Disasters.csv',delimiter=';')


In [4]:
df_events

,Dis No,Year,Seq,Glide,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Event Name,...,"Reconstruction Costs, Adjusted ('000 US$)",Insured Damages ('000 US$),"Insured Damages, Adjusted ('000 US$)",Total Damages ('000 US$),"Total Damages, Adjusted ('000 US$)",CPI,Adm Level,Admin1 Code,Admin2 Code,Geo Locations
0,1900-9002-CPV,1900,9002,NaN,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"2,849084409",NaN,NaN,NaN,NaN
1,1900-9001-IND,1900,9001,NaN,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"2,849084409",NaN,NaN,NaN,NaN
2,1902-0012-GTM,1902,12,NaN,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,...,NaN,NaN,NaN,25000.0,843726.0,"2,963047785",NaN,NaN,NaN,NaN
3,1902-0003-GTM,1902,3,NaN,Natural,Geophysical,Volcanic activity,Ash fall,NaN,Santa Maria,...,NaN,NaN,NaN,NaN,NaN,"2,963047785",NaN,NaN,NaN,NaN
4,1902-0010-GTM,1902,10,NaN,Natural,Geophysical,Volcanic activity,Ash fall,NaN,Santa Maria,...,NaN,NaN,NaN,NaN,NaN,"2,963047785",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16540,2023-0002-PHL,2023,2,NaN,Natural,Hydrological,Flood,NaN,NaN,NaN,...,NaN,NaN,NaN,12000.0,NaN,NaN,1;2,2357;2361;2362;2364;2368;67159;67162;67165;67166,24267;24272;67174,"Region II (Cagayan Valley), Region III (Centra..."
16541,2023-0056-ZMB,2023,56,NaN,Natural,Hydrological,Flood,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,65228;65231;65236,"Choma, Kalomo, Namwala (Adm2)."
16542,2023-0046-NZL,2023,46,NaN,Natural,Hydrological,Flood,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,2171,NaN,Auckland (Adm1).
16543,2023-0032-USA,2023,32,NaN,Natural,Meteorological,Storm,Convective storm,Tornado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,3214;3224,NaN,"Alabama, Georgia (Adm1)."


In [5]:
# Drop all events from before 1960
df_events = df_events[df_events["Year"] >= 1960]

In [6]:
# Drop disasters with certain subgroups that aren't relevant to the scope
df_events = df_events[df_events["Disaster Subgroup"] != "Biological"]
df_events = df_events[df_events["Disaster Subgroup"] != "Extra-terrestrial"]

In [7]:
# Returns ISO-3 code for a given country name
def get_country_code(c_name):
    country = pycountry.countries.get(name=c_name)
    code = None
    if country is not None:
        code = country.alpha_3
    return code

In [8]:
# Fill in missing ISO3 codes
df_location_data.loc[df_location_data["iso3"].isna(), "iso3"] = df_location_data[df_location_data["iso3"].isna()].apply(lambda x: get_country_code(x["country"]),axis=1)

In [9]:
# Fill in missing ISO3 codes not included in the Pycountry library
df_location_data.loc[df_location_data["country"] == "Palestina", "iso3"] = "PSE"
df_location_data.loc[df_location_data["country"] == "Virgin Islands; U.S.", "iso3"] = "VIR"
df_location_data.loc[df_location_data["country"] == "Reunion", "iso3"] = "REU"
df_location_data.loc[df_location_data["country"] == "Saint Helena", "iso3"] = "SHN"
df_location_data.loc[df_location_data["country"] == "Micronesia", "iso3"] = "FSM"
df_location_data.loc[df_location_data["country"] == "British Virgin Islands", "iso3"] = "VGB"

In [10]:
# Edit disasterno to be the same as in df_events
df_location_data["disasterno"] = df_location_data["disasterno"] + "-" + df_location_data["iso3"]

In [11]:
def extract_lat_long(row, key):
    location_data = df_location_data[df_location_data["disasterno"] == row["Dis No"]]
    if not location_data.empty:
        return location_data.iloc[0, :][key]
    else:
        return None

In [12]:
# Augment Latitude info
df_events["Latitude"] = df_events.apply(lambda row: extract_lat_long(row, "latitude"), axis=1)

# Augment Longitude info
df_events["Longitude"] = df_events.apply(lambda row: extract_lat_long(row, "longitude"), axis=1)

In [13]:
# Extract trajectory of disaster
def extract_trajectory(row):
    location_data = df_location_data[df_location_data["disasterno"] == row["Dis No"]]
    trajectory = []
    if not location_data.empty:
        for _, row in location_data.iterrows():
            trajectory.append((row["latitude"], row["longitude"]))
        return trajectory
    else:
        return None

In [14]:
# Augment trajectory info
df_events["Trajectory"] = df_events.apply(lambda row: extract_trajectory(row), axis=1)

In [15]:
df_events = df_events.drop(["Year", "Seq", "Glide", "Disaster Group", "Location", "OFDA Response", "Appeal", "Declaration", "Adm Level", "Admin1 Code", "Admin2 Code", "Geo Locations"], axis=1)

In [16]:
df_events

,Dis No,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Event Name,Country,ISO,Region,Continent,...,No Homeless,Total Affected,Reconstruction Costs ('000 US$),"Reconstruction Costs, Adjusted ('000 US$)",Insured Damages ('000 US$),"Insured Damages, Adjusted ('000 US$)",Total Damages ('000 US$),"Total Damages, Adjusted ('000 US$)",CPI,Trajectory
303,1960-0013-CHL,Geophysical,Earthquake,Tsunami,NaN,NaN,Chile,CHL,South America,Americas,...,NaN,2003000.0,NaN,NaN,NaN,NaN,550000.0,5442439.0,"10,10576299","[(-41.47604118, -72.81041733), (-39.87852955, ..."
304,1960-0026-AIA,Meteorological,Storm,Tropical cyclone,NaN,Donna,Anguilla,AIA,Caribbean,Americas,...,1000.0,1250.0,NaN,NaN,NaN,NaN,35000.0,346337.0,"10,10576299",None
305,1960-0025-ANT,Meteorological,Storm,Tropical cyclone,NaN,Donna,Netherlands Antilles,ANT,Caribbean,Americas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"10,10576299",None
306,1960-0024-ATG,Meteorological,Storm,Tropical cyclone,NaN,Donna,Antigua and Barbuda,ATG,Caribbean,Americas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"10,10576299",None
307,1960-0030-BGD,Meteorological,Storm,Tropical cyclone,NaN,NaN,Bangladesh,BGD,Southern Asia,Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"10,10576299","[(22.69803384, 91.11882816)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16540,2023-0002-PHL,Hydrological,Flood,NaN,NaN,NaN,Philippines (the),PHL,South-Eastern Asia,Asia,...,NaN,2000011.0,NaN,NaN,NaN,NaN,12000.0,NaN,NaN,None
16541,2023-0056-ZMB,Hydrological,Flood,NaN,NaN,NaN,Zambia,ZMB,Eastern Africa,Africa,...,NaN,22000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
16542,2023-0046-NZL,Hydrological,Flood,NaN,NaN,NaN,New Zealand,NZL,Australia and New Zealand,Oceania,...,1614.0,5400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
16543,2023-0032-USA,Meteorological,Storm,Convective storm,Tornado,NaN,United States of America (the),USA,Northern America,Americas,...,NaN,49.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [ ]:
df_events.to_csv('../Preprocessed-Natural-Disasters.csv', sep=";", index=False)

In [18]:
df_events[df_events['Longitude'].isnull() & df_events['Latitude'].isnull()][['Country', 'Start Year']].head(15)

,Country,Start Year
304,Anguilla,1960
305,Netherlands Antilles,1960
306,Antigua and Barbuda,1960
309,Bangladesh,1960
312,Hong Kong,1960
314,Bangladesh,1961
316,Bangladesh,1961
318,Canada,1961
320,France,1961
321,Greece,1961


In [19]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
    
# invert the dictionary
abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))

In [30]:
geolocator = Nominatim(user_agent='geoapiExercises')
def lat_long_to_state(lat,long):
    try:
        location = geolocator.reverse(f'{lat}, {long}')
    except:
        return None
    if (not location):
        return None
    address = location.raw['address']
    zip_code = address.get('postcode')
    state = address.get('state')
    if (state):
        return state
    if (not zip_code):
        return None
    
    if (len(zip_code) == 5 or len(zip_code) == 10):
        abrev = zipcodes.matching(f'{zip_code}')
        if (len(abrev) > 0):
            abrev = abrev[0]['state']
            return abbrev_to_us_state[abrev]
    return None

In [31]:
df_events['us state'] = df_events.apply(lambda row: lat_long_to_state(row['Latitude'],row['Longitude']), axis=1)

10310
30403
30512
21011
80303
13022
28059
13022
13130
49500
72200
25167
34760
16450
50470
04390
16450
98822
34270
04390
11701
11701
60504
45502
37347
10310
98816
98811
33315
47748
48600
56600
33315
04390
16450
16450
48600
38300
22203
76873
76873
06023
25800
76873
78387
02100
33553
76873
34270
76873
76873
21012
60504
30512
70503
11701
30512
30508
70503
20097
55902
25167
04390
25167
28059
72300
76873
76873
06023
76873
97064
30403
33315
13130
07046
50102
33315
14250
13022
30403
10310
13130
28059
50302
37347
13022
34270
15113
33553
78100
21012
10813
37347
98870
72200
53220
33315
33315
52152
13022
16450
82300
33315
38300
48700
97064
76111
78492
50905
33517
13022
13022
04390
28059
70503
70503
10969
25350
10310
50470
72200
34045
76873
00848
50470
75654
98800
62700
47748
30512
70503
60603
33315
37347
10310
33315
30601
80047
32350
35460
50470
71000
33553
32350
83200
04400
34235
15113
50701
66289
12099
10969
31103
25167
72200
22203
37565
81630
22203
25167
78201
51449
20503
11701
50701
50504
5040